In [2]:
# 2D Crack propagation in a bended beam

 - @todo: Plot the residual value
 - @todo: State dependencies - recalculate upon material parameter change

In [3]:
%matplotlib widget
import bmcs_utils.api as bu
from bmcs_shear.shear_crack.crack_propagation import CrackPropagation
ce = CrackPropagation(n_seg=250)
ce.sz_bd.cmm.trait_set(tau_1 = 4, s_1 = 0.2, tau_2 = 3, tau_3 = 2, s_2 = 1.4, s_3 = 5)
ce.sz_bd.cmm.trait_set(f_c = 33.3, f_t = 0.3 * 33.3 **(2/3), d_a = 16)
ce.sz_bd.Rectangle = True
ce.sz_bd.trait_set(H=600, B=250, L=3870)
ce.sz_cp.trait_set(x_00=ce.sz_bd.L * 0.3, n_m = 15)
ce.sz_cp.sz_ctr.trait_set(ell = 2)
ce.sz_bd.cross_section_layout.reinforcement.z_j[0] = 30
ce.sz_bd.cross_section_layout.reinforcement.A_j[0] = 615.44
ce.sz_bd.cross_section_layout.reinforcement.E_j[0] = 210000 / 100

\begin{align}
\sigma_\mathrm{F} = E w / L_\mathrm{eff}
\end{align}

In [4]:
bu.InteractiveWindow([ce, ce.sz_bd, ce.sz_bd.cross_section_layout, 
                      ce.sz_stress_profile, 
                      ce.sz_bd.cmm, ce.sz_bd.smm, ce.sz_cp, ce.sz_cp.sz_ctr]).interact()

<lambdifygenerated-24>:2: RuntimeWarning: invalid value encountered in sqrt
  return (select([less_equal(w, 0),True], [0,0.25*f_c*s*(-sqrt(2)*sqrt(w/d_a) + 1)*((2.44 - 39.04/f_c)*abs(s/w)**3 + 9.8/f_c)/(w*(s**4*(2.44 - 39.04/f_c)/w**4 + 1))], default=nan))


In [5]:
import numpy as np
S_Lb = ce.sz_stress_profile.S_Lb
S_La = ce.sz_stress_profile.S_La
idx = np.argmax(S_Lb[...,0])

In [6]:
S_Lb[idx,:], S_La[idx,:]

(array([7.74426492e+02, 4.57123369e-02]),
 array([7.74426492e+02, 4.57123369e-02]))

In [7]:
ce.sz_cp.T_Mab[idx,...]

array([[1., 0.],
       [0., 1.]])

In [8]:
np.einsum('i,i', S_Lb[idx,:], S_Lb[idx,:])

599736.3942562717

In [9]:
np.einsum('i,i', S_La[idx,:], S_La[idx,:])

599736.3942562715

Todo:

 - include the reinforcement material law - partially done 
   (but for strain-based material law) - replace with the crack bridge law
 - transformation of the crack bridging force to the global coordinates
 - include the dowel effect 

In [10]:
#ce.sz_cp._get_norm_n_vec_L()

In [11]:
#ce.sz_cp._get_x_Ka()

In [12]:
#ce.sz_stress_profile._get_S_La()

In [13]:
#ce.sz_stress_profile._get_F_La()

In [14]:
#ce.sz_stress_profile._get_F_ag()

In [15]:
#ce.sz_stress_profile._get_u_Lb()

In [16]:
#ce.sz_stress_profile._get_S_Lb()[:,1]

In [17]:
ce.sz_stress_profile.u_La[...,1]

array([ 3.85459969e-06,  3.80843682e-06,  3.76227398e-06,  3.71611111e-06,
        3.66994821e-06,  3.62378534e-06,  3.57762247e-06,  3.53145960e-06,
        3.48529674e-06,  3.43913387e-06,  3.39297100e-06,  3.34680810e-06,
        3.30064523e-06,  3.25448238e-06,  3.20319030e-06,  3.14676902e-06,
        3.09034774e-06,  3.03392646e-06,  2.97750516e-06,  2.92108388e-06,
        2.86466259e-06,  2.80824130e-06,  2.75182002e-06,  2.69539873e-06,
        2.63897745e-06,  2.58255616e-06,  2.52613486e-06,  2.46971358e-06,
        2.41329229e-06,  2.35687101e-06,  2.30044972e-06,  2.24402844e-06,
        2.18760715e-06,  2.13118587e-06,  2.07476458e-06,  2.01834330e-06,
        1.96192201e-06,  1.90550072e-06,  1.84907944e-06,  1.79265815e-06,
        1.73623687e-06,  1.67981557e-06,  1.62339428e-06,  1.56697301e-06,
        1.51055173e-06,  1.45413043e-06,  1.39770914e-06,  1.34128786e-06,
        1.28486657e-06,  1.22844529e-06,  1.17202399e-06,  1.11560270e-06,
        1.05918143e-06,  

In [18]:
ce.sz_cp.x_Ka

array([[1161.        ,    0.        ],
       [1161.        ,    3.57142857],
       [1161.        ,    7.14285714],
       [1161.        ,   10.71428571],
       [1161.        ,   14.28571429],
       [1161.        ,   17.85714286],
       [1161.        ,   21.42857143],
       [1161.        ,   25.        ],
       [1161.        ,   28.57142857],
       [1161.        ,   32.14285714],
       [1161.        ,   35.71428571],
       [1161.        ,   39.28571429],
       [1161.        ,   42.85714286],
       [1161.        ,   46.42857143],
       [1161.        ,   50.        ],
       [1161.        ,   54.36507937],
       [1161.        ,   58.73015873],
       [1161.        ,   63.0952381 ],
       [1161.        ,   67.46031746],
       [1161.        ,   71.82539683],
       [1161.        ,   76.19047619],
       [1161.        ,   80.55555556],
       [1161.        ,   84.92063492],
       [1161.        ,   89.28571429],
       [1161.        ,   93.65079365],
       [1161.        ,   

In [19]:
ce.sz_stress_profile.u_La[...,1]

array([ 3.85459969e-06,  3.80843682e-06,  3.76227398e-06,  3.71611111e-06,
        3.66994821e-06,  3.62378534e-06,  3.57762247e-06,  3.53145960e-06,
        3.48529674e-06,  3.43913387e-06,  3.39297100e-06,  3.34680810e-06,
        3.30064523e-06,  3.25448238e-06,  3.20319030e-06,  3.14676902e-06,
        3.09034774e-06,  3.03392646e-06,  2.97750516e-06,  2.92108388e-06,
        2.86466259e-06,  2.80824130e-06,  2.75182002e-06,  2.69539873e-06,
        2.63897745e-06,  2.58255616e-06,  2.52613486e-06,  2.46971358e-06,
        2.41329229e-06,  2.35687101e-06,  2.30044972e-06,  2.24402844e-06,
        2.18760715e-06,  2.13118587e-06,  2.07476458e-06,  2.01834330e-06,
        1.96192201e-06,  1.90550072e-06,  1.84907944e-06,  1.79265815e-06,
        1.73623687e-06,  1.67981557e-06,  1.62339428e-06,  1.56697301e-06,
        1.51055173e-06,  1.45413043e-06,  1.39770914e-06,  1.34128786e-06,
        1.28486657e-06,  1.22844529e-06,  1.17202399e-06,  1.11560270e-06,
        1.05918143e-06,  

In [20]:
ce.sz_stress_profile.normalized_def #[...,1]

AttributeError: 'SZStressProfile' object has no attribute 'normalized_def'

In [21]:
ce.sz_stress_profile.tau_exp

AttributeError: 'SZStressProfile' object has no attribute 'tau_exp'

In [ ]:
ce.sz_stress_profile.F_Na[...,0]

In [ ]:
ce.sz_ctr._get_x_rot_ak()